In [7]:
# Import Pandas to provide DataFrame support
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Import BeautifulSoup
from bs4 import BeautifulSoup

import numpy as np # library to handle data in a vectorized manner

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


In [8]:
# Set up Foursquare
CLIENT_ID = 'K2D5QVKQ0QPUPIKECQU2JIB3CHVFT4ZMTJQC5WVTRNYTDD5Z'
CLIENT_SECRET = '5ZVFLDC04W0W5SHCBZ11MDRG0MELPIER3QIFUOAWFAIZKJNK'
VERSION = '20200808' # Foursquare API version
LIMIT = 300
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: K2D5QVKQ0QPUPIKECQU2JIB3CHVFT4ZMTJQC5WVTRNYTDD5Z
CLIENT_SECRET:5ZVFLDC04W0W5SHCBZ11MDRG0MELPIER3QIFUOAWFAIZKJNK


In [9]:
# Use the Requests get method to request the top sites in Chicago
page = requests.get(
    "https://foursquare.com/explore?cat=topPicks&mode=url&near=London%2C%20Greater%20London%2C%20United%20Kingdom&nearGeoId=72057594040571679")

# Convert the HTML response into a BeautifulSoup Object
soup = BeautifulSoup(page.content, 'html.parser')

# Use the BeautifulSoup find_all method to extract each top site venue details.
top_venues = soup.find_all('div', class_='venueDetails')
top_venues

[<div class="venueDetails"><div class="venueScore positive" style="background-color: #00B551;" title="9.7/10 - People like this place">9.7</div><div class="venueName"><h2><span class="venueIndex">1</span>. <a href="/v/hampstead-heath/4ac518cef964a5201aa620e3" target="_blank">Hampstead Heath</a></h2></div><div class="venueMeta"><div class="venueAddressData"><div class="venueData"><span class="venueDataItem"><span class="categoryName">Park</span><span class="delim"> • </span></span></div><div class="venueAddress">E Heath Rd, London</div></div></div></div>,
 <div class="venueDetails"><div class="venueScore positive" style="background-color: #00B551;" title="9.7/10 - People like this place">9.7</div><div class="venueName"><h2><span class="venueIndex">2</span>. <a href="/v/regents-park/4b233922f964a520785424e3" target="_blank">Regent's Park</a></h2></div><div class="venueMeta"><div class="venueAddressData"><div class="venueData"><span class="venueDataItem"><span class="categoryName">Park</s

In [10]:
venue_columns = ['id', 
                 'score', 
                 'category', 
                 'name', 
                 'address',
                 'postalcode',
                 'city',
                 'href', 
                 'latitude', 
                 'longitude']

# Create the empty top venues dataframe
df_top_venues = pd.DataFrame(columns=venue_columns)

# For each venue in the BeautifulSoup HTML object
for venue in top_venues:
    
    # Extract the available attributes
    venue_name = venue.find(target="_blank").get_text()
    venue_score = venue.find(class_="venueScore positive").get_text()
    venue_cat = venue.find(class_="categoryName").get_text()
    venue_href = venue.find(class_="venueName").h2.a['href']
    venue_id = venue_href.split('/')[-1]

    if 'promotedTipId' in venue_id: 
        continue
        
    # Contruct the FourSquare venue API URL
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    
    # Request the venue data
    result = requests.get(url).json()
        
   

In [12]:
result

{'meta': {'code': 429,
  'errorType': 'quota_exceeded',
  'errorDetail': 'Quota exceeded',
  'requestId': '5ed7c5af618f43001b33a68e'},
 'response': {}}

In [12]:
# tranform venues into a dataframe
#dataframe = json_normalize(result)
#dataframe.head()

/Users/joseph_arber/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,meta.code,meta.requestId,response.venue.id,response.venue.name,response.venue.location.lat,response.venue.location.lng,response.venue.location.labeledLatLngs,response.venue.location.cc,response.venue.location.city,response.venue.location.state,response.venue.location.country,response.venue.location.formattedAddress,response.venue.canonicalUrl,response.venue.categories,response.venue.verified,response.venue.stats.tipCount,response.venue.likes.count,response.venue.likes.groups,response.venue.likes.summary,response.venue.dislike,response.venue.ok,response.venue.rating,response.venue.ratingColor,response.venue.ratingSignals,response.venue.beenHere.count,response.venue.beenHere.unconfirmedCount,response.venue.beenHere.marked,response.venue.beenHere.lastCheckinExpiredAt,response.venue.specials.count,response.venue.specials.items,response.venue.photos.count,response.venue.photos.groups,response.venue.reasons.count,response.venue.reasons.items,response.venue.hereNow.count,response.venue.hereNow.summary,response.venue.hereNow.groups,response.venue.createdAt,response.venue.tips.count,response.venue.tips.groups,response.venue.shortUrl,response.venue.timeZone,response.venue.listed.count,response.venue.listed.groups,response.venue.popular.isOpen,response.venue.popular.isLocalHoliday,response.venue.popular.timeframes,response.venue.seasonalHours,response.venue.pageUpdates.count,response.venue.pageUpdates.items,response.venue.inbox.count,response.venue.inbox.items,response.venue.attributes.groups,response.venue.bestPhoto.id,response.venue.bestPhoto.createdAt,response.venue.bestPhoto.source.name,response.venue.bestPhoto.source.url,response.venue.bestPhoto.prefix,response.venue.bestPhoto.suffix,response.venue.bestPhoto.width,response.venue.bestPhoto.height,response.venue.bestPhoto.visibility,response.venue.colors.highlightColor.photoId,response.venue.colors.highlightColor.value,response.venue.colors.highlightTextColor.photoId,response.venue.colors.highlightTextColor.value,response.venue.colors.algoVersion
0,200,5ed7c0d19c6f59001b2f388c,4b717344f964a52043472de3,Richmond Riverside,51.458298,-0.306895,"[{'label': 'display', 'lat': 51.45829762937411...",GB,Richmond,Greater London,United Kingdom,"[Richmond, Greater London, United Kingdom]",https://foursquare.com/v/richmond-riverside/4b...,"[{'id': '56aa371be4b08b9a8d5734c3', 'name': 'W...",False,14,194,"[{'type': 'others', 'count': 194, 'items': []}]",194 Likes,False,False,9.4,00B551,211,0,0,False,0,0,[],321,"[{'type': 'venue', 'name': 'Venue photos', 'co...",1,"[{'summary': 'Lots of people like this place',...",0,Nobody here,[],1265726276,14,"[{'type': 'others', 'name': 'All tips', 'count...",http://4sq.com/cKPeg4,Europe/London,36,"[{'type': 'others', 'name': 'Lists from other ...",False,False,"[{'days': 'Today', 'includesToday': True, 'ope...",[],0,[],0,[],[],4f365419e4b0571dc63976c2,1328960537,Foursquare for iOS,https://foursquare.com/download/#/iphone,https://fastly.4sqi.net/img/general/,/c7ofO5pHf8FpZdx3ruJYviPVB9DLemIh5mTSFyWMi20.jpg,720,502,public,4f365419e4b0571dc63976c2,-14147584,4f365419e4b0571dc63976c2,-1,3


In [11]:
# Get the properly formatted address and the latitude and longitude
venue_address = result['response']['venue']['location']['formattedAddress']
venue_city = result['response']['venue']['location']['city']
venue_latitude = result['response']['venue']['location']['lat']
venue_longitude = result['response']['venue']['location']['lng']
    
    # Add the venue to the top venues dataframe
df_top_venues = df_top_venues.append({'id': venue_id,
                                          'score': venue_score,
                                          'category': venue_cat,
                                          'name': venue_name,
                                          'address': venue_address,
                                          'city': venue_city,
                                          'href': venue_href,
                                          'latitude': venue_latitude,
                                          'longitude': venue_longitude}, ignore_index=True)

KeyError: 'venue'

In [20]:
df_top_venues

,id,score,category,name,address,postalcode,city,href,latitude,longitude
0,4b717344f964a52043472de3,9.4,Waterfront,Richmond Riverside,"[Richmond, Greater London, United Kingdom]",NaN,Richmond,/v/richmond-riverside/4b717344f964a52043472de3,51.458298,-0.306895


In [22]:
df_top_venues.shape

(1, 10)